### Hàm đọc file txt để lấy thông tin tọa độ bounding box thực tế

In [1]:
def load_bounding_boxes_from_txt(file_path):
    ground_truths = []
    with open(file_path, "r") as f:
        for line in f.readlines():
            values = line.strip().split()
            class_id = int(values[0])
            x, y, w, h = map(float, values[1:])
            ground_truths.append({"class_id": class_id, "bbox": [x, y, w, h]})
    return ground_truths

# test = load_bounding_boxes_from_txt('images/input_images/00023.txt')
# print(test[0]['bbox'])


### Hàm tính IoU giữa bounding box dự đoán và bounding box thực tế

In [2]:
def calculate_iou(box1, box2, image_width, image_height):
    # Convert normalized values to pixel values
    x1_min = (box1[0] - box1[2] / 2) * image_width
    y1_min = (box1[1] - box1[3] / 2) * image_height
    x1_max = (box1[0] + box1[2] / 2) * image_width
    y1_max = (box1[1] + box1[3] / 2) * image_height
    
    x2_min = (box2[0] - box2[2] / 2) * image_width
    y2_min = (box2[1] - box2[3] / 2) * image_height
    x2_max = (box2[0] + box2[2] / 2) * image_width
    y2_max = (box2[1] + box2[3] / 2) * image_height
    
    # Calculate intersection
    x_intersection_min = max(x1_min, x2_min)
    y_intersection_min = max(y1_min, y2_min)
    x_intersection_max = min(x1_max, x2_max)
    y_intersection_max = min(y1_max, y2_max)
    
    # If there's no intersection
    if x_intersection_min >= x_intersection_max or y_intersection_min >= y_intersection_max:
        return 0.0
    
    # Calculate the area of the intersection
    intersection_area = (x_intersection_max - x_intersection_min) * (y_intersection_max - y_intersection_min)
    
    # Calculate the area of both bounding boxes
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    
    # Calculate the area of the union
    union_area = box1_area + box2_area - intersection_area
    
    # Calculate IoU
    iou = intersection_area / union_area
    return iou

# Ví dụ sử dụng:
# box1 = [0.81796587, 0.686112, 0.03207427, 0.047852866]
# box2 = [0.8158088235294118, 0.628125, 0.03602941176470588, 0.06125]
# image = cv2.imread('images/input_images/00021.jpg')
# image_width = image.shape[1]  # Kích thước chiều rộng ảnh
# image_height = image.shape[0]  # Kích thước chiều cao ảnh
# print(image.shape[1] )
# iou = calculate_iou(box2, box1, image_width, image_height)
# print(f"IoU: {iou}")


### Hàm tính chỉ số mAP (mean average precision)

In [3]:
from collections import defaultdict
from sklearn.metrics import precision_recall_curve, auc

def compute_map(detections, ground_truths, class_names, iou_threshold=0.5):
    # Lưu kết quả tính Precision và Recall
    ap_per_class = {}
    total_detections = 0
    results_per_class = defaultdict(lambda: {"TP": 0, "FP": 0, "confidence": [], "iou": []})

    for class_id in range(len(class_names)):
        gt_class = ground_truths[class_id]
        pred_class = detections[class_id]

        matched = set()
        for pred in pred_class:
            iou_max = 0
            gt_match = None
            for i, gt in enumerate(gt_class):
                iou = calculate_iou(pred["bbox"], gt["bbox"], pred["img_width"], pred["img_height"])
                if iou > iou_max:
                    iou_max = iou
                    gt_match = i
            if iou_max >= iou_threshold and gt_match not in matched:
                results_per_class[class_id]["TP"] += 1
                matched.add(gt_match)
            else:
                results_per_class[class_id]["FP"] += 1

            results_per_class[class_id]["confidence"].append(pred["confidence"])
            results_per_class[class_id]["iou"].append(iou_max)

        total_detections += len(pred_class)

        # Tính Precision và Recall
        TP = results_per_class[class_id]["TP"]
        FP = results_per_class[class_id]["FP"]
        conf = results_per_class[class_id]["confidence"]
        precisions, recalls, _ = precision_recall_curve([1] * TP + [0] * FP, conf)
        ap_per_class[class_id] = auc(recalls, precisions) * 100

    # Tính mAP
    map_value = sum(ap_per_class.values()) / len(class_names)
    
    # Trả về TP, FP, và mAP
    return results_per_class, ap_per_class, map_value, total_detections


In [4]:
import numpy as np
import cv2
import os

def load_images_from_folder(folder):
    images = []
    filenames = []  
    for filename in os.listdir(folder):
        if filename.endswith(".jpg"):  
            img = cv2.imread(os.path.join(folder, filename))
            if img is not None:
                images.append(img)
                filenames.append(filename)
    return images, filenames

def main():
    input_folder_path = "images/input_images"
    output_folder_path = "images/output_images"
    all_images, filenames = load_images_from_folder(input_folder_path)

    ground_truths = defaultdict(list)
    detections = defaultdict(list)

    net = cv2.dnn.readNet("yolov3_training_last.weights", "yolov3_training.cfg")

    classes = []
    with open("signs.names.txt", "r") as f:
        classes = [line.strip() for line in f.readlines()]

    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    confidence_threshold = 0.5
    font = cv2.FONT_HERSHEY_SIMPLEX

    iou_per_class = {class_id: [] for class_id in range(len(classes))}  # Khởi tạo dictionary để lưu IoU theo class

    for i, img in enumerate(all_images):
        k= 0
        filename = filenames[i]
        ground_truth = load_bounding_boxes_from_txt('images/input_images/' + os.path.splitext(filename)[0] + '.txt')
        height, width, channels = img.shape

        # Detecting objects (YOLO)
        blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        outs = net.forward(output_layers)

        class_ids = []
        confidences = []
        boxes = []
        annotation_predicts = []
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > confidence_threshold:
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)
                    boxes.append([x, y, w, h])
                    annotation_predicts.append([detection[0], detection[1], detection[2], detection[3]])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)
                    detections[class_id].append({"bbox": [detection[0], detection[1], detection[2], detection[3]],
                                             "confidence": confidence,
                                             "img_width": width, "img_height": height})

            for gt in ground_truth:
                class_id = gt["class_id"]
                ground_truths[class_id].append({"bbox": gt["bbox"], "img_width": width, "img_height": height})

        # NMS loại bỏ overlap
        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

        for j in indexes:
                x, y, w, h = boxes[j]
                iou = calculate_iou(ground_truth[k]['bbox'], annotation_predicts[j], width, height)
                iou_per_class[class_ids[j]].append(iou)
                k=k+1
                label = str(classes[class_ids[j]]) + "=" + str(round(confidences[j] * 100, 2)) + "%, IoU = " + str(round(iou, 2))
                img = cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
                img = cv2.putText(img, label, (x, y - 10), font, 0.5, (255, 0, 0), 2)

        # Ghi file output với tên mới
        # base_name = os.path.splitext(filename)[0]  # Lấy tên file không có đuôi
        # output_file_path = os.path.join(output_folder_path, f"{base_name}_prediction.jpg")
        # cv2.imwrite(output_file_path, img)
    
    # Tính trung bình IoU theo class
    average_iou_per_class = {class_id: np.mean(iou_list) if iou_list else 0 for class_id, iou_list in iou_per_class.items()}
    for class_id, avg_iou in average_iou_per_class.items():
        print(f"Class {classes[class_id]} ({class_id}): Average IoU = {avg_iou:.2f}")

    print('-----------------')

     # Tính toán mAP
    results_per_class, ap_per_class, map_value, total_detections = compute_map(detections, ground_truths, classes)
    print(f"Total number of detections: {total_detections}")
    for class_id, ap in ap_per_class.items():
        print(f"{classes[class_id]}, AP = {ap:.2f}%, TP = {results_per_class[class_id]['TP']}, FP = {results_per_class[class_id]['FP']}")
    print(f"mAP = {map_value:.2f}%")

main()


Class prohibitory (0): Average IoU = 0.88
Class danger (1): Average IoU = 0.86
Class mandatory (2): Average IoU = 0.78
Class other (3): Average IoU = 0.81
-----------------
Total number of detections: 29
prohibitory, AP = 75.49%, TP = 6, FP = 4
danger, AP = 37.79%, TP = 5, FP = 4
mandatory, AP = 41.67%, TP = 2, FP = 2
other, AP = 34.44%, TP = 3, FP = 3
mAP = 47.35%
